In [2]:
from get_cosine_sim import calculate_cosine, blending
import pandas as pd
import json

In [3]:
reviews=pd.read_csv('user_review_with_LLM.csv')
# item=pd.read_csv('item.csv')
item=pd.read_csv('item2.csv')
# test=pd.read_csv('graph_test.csv')
test=pd.read_csv('graph2_test.csv')

In [4]:
with open('top_k_predictions_rating.json', 'r') as json_file:
    ngcf_rating = json.load(json_file)
with open('top_k_predictions_sentiment.json', 'r') as json_file:
    ngcf_sent = json.load(json_file)
with open('top_k_predictions_rating_sentiment.json', 'r') as json_file:
    ngcf_blend = json.load(json_file)

In [5]:
# evaluation function
def calculate_recall (final_prediction, test):
    gt_user, gt_item = list(test['user_id']), list(test['item_id'])
    
    cnt=0

    for i in range(test.shape[0]):
        user=gt_user[i]
        if gt_item[i] in final_prediction[user]:
            cnt+=1
    
    correct = cnt
    gt = test.shape[0]
    recall = correct/gt
    
    return recall, correct, gt

def calculate_map (k, final_prediction, test):
    n=test['user_id'].nunique()
    ap_list=[]
    gt = [(row.user_id, row.item_id) for row in test.itertuples(index=False)]
    for user in final_prediction.keys():
        cnt=0 #정답 세기
        user_sum=0
        for idx, item in enumerate(final_prediction[user]):
            if (user, item) in gt:
                cnt+=1
            user_sum+=(cnt/(idx+1))
        ap_list.append(user_sum/k)
    
    return sum(ap_list)/n

In [6]:
with open('evaluation_result.txt', 'w') as file:
    file.write("evaluation result\n\n")

In [7]:
with open('evaluation_result.txt', 'a') as file:
    file.write("Baseline:\n")

In [12]:
for ngcf, edge in edges:
    for k in K:
        beta=1
        n=100
        print(n, k, beta)
        final_prediction=blending(n, k, beta, ngcf, {}) 
        recall, recall_correct, recall_gt = calculate_recall(final_prediction, test)
        MAP = calculate_map(k, final_prediction, test)
        with open('evaluation_result.txt', 'a') as file:
            file.write(f"edge: {edge}, k: {k}, -> recall: {recall}, map: {MAP}\n")

100 50 1
100 100 1
100 50 1
100 100 1
100 50 1
100 100 1


In [13]:
with open('evaluation_result.txt', 'a') as file:
    file.write("\nWith NLP:\n")

In [6]:
# setting parameters
edges=[(ngcf_blend, 'blend')]
N=[100, 500, 1000]
K=[50, 100]
Beta=[0.25, 0.5, 0.75] #beta=1 -> blend 없음
modes=['review']

In [7]:
from get_cosine_sim import calculate_cosine, blending

for ngcf, edge in edges:
    for n in N:
        for k in K:
            for beta in Beta:
                for mode in modes:
                    sim_cache=calculate_cosine(mode, n, ngcf, reviews, item)
                    final_prediction=blending(n, k, beta, ngcf, sim_cache) 
                    
                    recall, recall_correct, recall_gt = calculate_recall(final_prediction, test)
                    MAP = calculate_map(k, final_prediction, test)
                    
                    with open('evaluation_result.txt', 'a') as file:
                        file.write(f"edge: {edge}, n: {n}, k: {k}, beta: {beta}, mode: review -> recall: {recall}, map: {MAP}\n")
                        file.write(f"edge: {edge}, n: {n}, k: {k}, beta: {beta}, mode: LLM -> recall: {recall}, map: {MAP}\n")

cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed
cosine sim is calculated
blending is completed


In [ ]:
with open('evaluation_result.txt', 'r') as file:
    content = file.read()
    print(content)

# Check the final result

In [26]:
baseline=[]
ours=[]
edge, rating, n, k, beta, mode, recall, MAP = [], [], [], [], [], [], [], []

with open('evaluation_result.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if ('edge' in line):
            line=line.replace(" -> ", ",")
            line=line.replace(",,", ", ")
            pairs = line.split(",")
            result_dict = {}

            for pair in pairs:
                key, value = pair.split(":")
                result_dict[key.strip()] = value.strip() 
                
            result_df = pd.DataFrame([result_dict])
            df = pd.concat([df, result_df], ignore_index=True)
                
            print(line.strip())

edge: rating, k: 50, recall: 0.009921377761138151, map: 0.00034914901797021455
edge: rating, k: 100, recall: 0.01497566454511419, map: 0.00031201081592421333
edge: sent, k: 50, recall: 0.01067016098839386, map: 0.00040268042563045204
edge: sent, k: 100, recall: 0.020217147135904157, map: 0.0003722130928257832
edge: blend, k: 50, recall: 0.0041183077499064025, map: 0.00011583982604682916
edge: blend, k: 100, recall: 0.01272931486334706, map: 0.00013808490723769837
edge: rating, n: 100, k: 50, beta: 0.25, mode: review,recall: 0.011793335829277425, map: 0.0007548054013371154
edge: rating, n: 100, k: 50, beta: 0.25, mode: LLM,recall: 0.011793335829277425, map: 0.0007548054013371154
edge: rating, n: 100, k: 50, beta: 0.5, mode: review,recall: 0.011606140022463497, map: 0.0005894006318777449
edge: rating, n: 100, k: 50, beta: 0.5, mode: LLM,recall: 0.011606140022463497, map: 0.0005894006318777449
edge: rating, n: 100, k: 50, beta: 0.75, mode: review,recall: 0.011606140022463497, map: 0.00049

In [43]:
for col in ['n', 'k', 'beta', 'recall', 'map']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [98]:
baseline=df2[:6]

In [99]:
pd.DataFrame({'recall': baseline.groupby('edge')['recall'].mean(), 'map': baseline.groupby('edge')['map'].mean()})

,recall,map
edge,,
blend,0.842381,0.012696
rating,1.244852,0.033058
sent,1.544365,0.038745


In [100]:
pd.DataFrame({'recall': df2.groupby('edge')['recall'].mean(), 'map': df2.groupby('edge')['map'].mean()})

,recall,map
edge,,
blend,1.909397,0.043203
rating,3.248832,0.120353
sent,3.085282,0.110818


In [101]:
pd.DataFrame({'recall': df2.groupby('n')['recall'].mean(), 'map': df2.groupby('n')['map'].mean()})

,recall,map
n,,
100.0,1.424768,0.054987
500.0,3.132410,0.102243
1000.0,3.942552,0.127693


In [102]:
pd.DataFrame({'recall': df2.groupby('beta')['recall'].mean(), 'map': df2.groupby('beta')['map'].mean()})

,recall,map
beta,,
0.25,3.388244,0.132048
0.50,2.844336,0.090389
0.75,2.267149,0.062485


In [53]:
df.groupby('mode')['recall'].mean(), df.groupby('mode')['map'].mean()

(mode
 LLM       0.028332
 review    0.028332
 Name: recall, dtype: float64,
 mode
 LLM       0.00095
 review    0.00095
 Name: map, dtype: float64)

In [57]:
df2=df.copy()
df2['recall']=df2['recall']*100
# df2=df2['recall']*100
df2['map']=df2['map']*100

In [93]:
# df2_edge=df2.groupby('edge')['recall'].mean()
pd.DataFrame({'recall': df2.groupby('edge')['recall'].mean(), 'map': df2.groupby('edge')['map'].mean()})

,recall,map
edge,,
blend,1.909397,0.043203
rating,3.248832,0.120353
sent,3.085282,0.110818


In [89]:
df2.groupby('edge')['recall'].mean(), df2.groupby('edge')['map'].mean()

edge
blend     1.909397
rating    3.248832
sent      3.085282
Name: recall, dtype: float64

In [60]:
df2.groupby('n')['recall'].mean(), df2.groupby('n')['map'].mean()

(n
 100.0     1.424768
 500.0     3.132410
 1000.0    3.942552
 Name: recall, dtype: float64,
 n
 100.0     0.054987
 500.0     0.102243
 1000.0    0.127693
 Name: map, dtype: float64)

In [61]:
df2.groupby('beta')['recall'].mean(), df2.groupby('beta')['map'].mean()

(beta
 0.25    3.388244
 0.50    2.844336
 0.75    2.267149
 Name: recall, dtype: float64,
 beta
 0.25    0.132048
 0.50    0.090389
 0.75    0.062485
 Name: map, dtype: float64)

In [62]:
df2.groupby('mode')['recall'].mean(), df2.groupby('mode')['map'].mean()

(mode
 LLM       2.833243
 review    2.833243
 Name: recall, dtype: float64,
 mode
 LLM       0.094974
 review    0.094974
 Name: map, dtype: float64)

In [64]:
df.to_csv('evaluation_result.csv', index=False)
df2.to_csv('evaluation_result_percent.csv', index=False)

# random evaluation

In [65]:
from get_cosine_sim import calculate_cosine, blending
import pandas as pd
import json

In [66]:
with open('ngcf_random.json', 'r') as json_file:
    ngcf_random = json.load(json_file)

In [73]:
rand=[(ngcf_random, 'random')]

In [69]:
with open('evaluation_result_random.txt', 'w') as file:
    file.write("random: result\n")

In [74]:
for ngcf, edge in rand:
    for k in K:
        beta=1
        n=100
        final_prediction=blending(n, k, beta, ngcf, {}) 
        recall, recall_correct, recall_gt = calculate_recall(final_prediction, test)
        MAP = calculate_map(k, final_prediction, test)
        with open('evaluation_result_random.txt', 'a') as file:
            file.write(f"edge: {edge}, k: {k}, -> recall: {recall}, map: {MAP}\n")

In [76]:
recall*100, MAP*100

(0.037439161362785474, 0.0006638083704802074)